In [1]:
import pandas as pd
import datetime as dt
from game_log import load_current_line
import matplotlib.pyplot as plt

today = dt.datetime.today()
year = today.year
month = today.month
day = today.day
today_str = f'{year}_{month}_{day}'

pp_path = f'Lines/pp/pp_{today_str}.csv'
unabated_path = f'Lines/unabated/unabated_{today_str}.csv'
dog_path = f'Lines/dog/doglines_{today.month}_{today.day}.csv'

pp_lines = load_current_line(pp_path,'prop_id','time')
unabated = load_current_line(unabated_path,'prop_id','time')
dog_lines = pd.read_csv(dog_path,index_col=0)

In [2]:
#pp_lines['date'] = pp_lines["date"].apply(lambda x: pd.to_datetime(x,utc=True))
#unabated["event_time"].apply(lambda x: pd.to_datetime(x,utc=True))

In [3]:
nba = pp_lines.loc[pp_lines.league_id==7]
combined = unabated.merge(pp_lines[['player','line','stat']],how='inner', on=['player','stat'],suffixes=['_u','_pp'])
combined

,prop_id,player,player_id,line_u,stat,league_id,event_time,opp,team,count,over_prob,under_prob,pred,time,line_pp
0,001cc313-b,Jordan Poole,43093,2.5,TRB,3,2023-11-10 18:00:00-06:00,CHA,WAS,12,0.496812,0.503188,2.659030,2023-11-10 13:03:00,2.5
1,a5babb15-8,Ivica Zubac,42545,10.5,PTS,3,2023-11-10 19:30:00-06:00,DAL,LAC,13,0.475486,0.524514,10.475731,2023-11-10 13:03:00,10.5
2,a5babb15-8,Ivica Zubac,42545,10.5,PTS,3,2023-11-10 19:30:00-06:00,DAL,LAC,13,0.475486,0.524514,10.475731,2023-11-10 13:03:00,6.0
3,a659a923-1,Ivica Zubac,42545,19.0,PTS+TRB,3,2023-11-10 19:30:00-06:00,DAL,LAC,1,0.500000,0.500000,19.519369,2023-11-10 13:03:00,19.0
4,a659a923-1,Ivica Zubac,42545,19.5,PTS+TRB,3,2023-11-10 19:30:00-06:00,DAL,LAC,6,0.492045,0.507955,19.519369,2023-11-10 13:03:00,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2079,550b0608-d,Dominic Lovett,185928,47.5,Receiving Yards,2,2023-11-11 18:00:00-06:00,MIS,UGA,2,0.497923,0.502077,49.174161,2023-11-10 13:03:00,50.5
2080,550b0608-d,Dominic Lovett,185928,50.5,Receiving Yards,2,2023-11-11 18:00:00-06:00,MIS,UGA,3,0.500000,0.500000,49.174161,2023-11-10 13:03:00,50.5
2081,550b0608-d,Dominic Lovett,185928,48.5,Receiving Yards,2,2023-11-11 18:00:00-06:00,MIS,UGA,3,0.500000,0.500000,49.174161,2023-11-10 13:03:00,50.5
2082,55212216-0,Donovan Edwards,66190,26.5,Rush Yards,2,2023-11-11 11:00:00-06:00,PSU,MICH,3,0.500000,0.500000,24.778067,2023-11-10 13:03:00,26.5


In [4]:
combined = combined[['prop_id', 'player','line_u','line_pp', 'player_id', 'stat', 
       'event_time', 'opp', 'team', 'count', 'over_prob', 'under_prob', 'pred',]]

In [5]:
combined.loc[combined.line_u == combined.line_pp].sort_values(by='over_prob').head(15)

,prop_id,player,line_u,line_pp,player_id,stat,event_time,opp,team,count,over_prob,under_prob,pred
892,bc62eda3-6,Alex Anzalone,0.25,0.25,34194,Sacks,2023-11-12 15:05:00-06:00,LAC,DET,1,0.294466,0.705534,0.347606
718,fd74cd08-7,Khalil Mack,0.25,0.25,35346,Sacks,2023-11-12 15:05:00-06:00,DET,LAC,2,0.457134,0.542866,0.609542
1910,474d57fd-4,Deni Avdija,6.50,6.50,43250,TRB,2023-11-10 18:00:00-06:00,CHA,WAS,12,0.460718,0.539282,6.424360
1577,1b1b32d0-a,Fletcher Loyer,9.50,9.50,257708,PTS,2023-11-10 18:00:00-06:00,MOR,PUR,1,0.460865,0.539135,9.373380
1337,3717d99b-8,Tobias Harris,0.50,0.50,5424,BLK,2023-11-10 18:00:00-06:00,DET,PHI,3,0.465066,0.534934,0.624061
1302,33831016-3,Santiago Vescovi,2.50,2.50,269557,3P,2023-11-10 20:00:00-06:00,WISC,TEN,1,0.466934,0.533066,2.540192
814,c62d9ada-2,Rhamondre Stevenson,3.50,3.50,38797,Receptions,2023-11-12 08:30:00-06:00,IND,NE,11,0.469213,0.530787,3.524615
1545,18f2f63d-a,Donta Scott,12.50,12.50,245818,PTS,2023-11-10 18:00:00-06:00,DAV,MD,1,0.470196,0.529804,12.412118
1681,6257b80c-7,Lauri Markkanen,3.50,3.50,43502,3P,2023-11-10 19:00:00-06:00,MEM,UTA,11,0.470538,0.529462,3.531385
953,d800ed55-7,Jonathan Allen,0.25,0.25,33952,Sacks,2023-11-12 15:25:00-06:00,SEA,WAS,2,0.471797,0.528203,0.636946


In [6]:
combined.loc[combined.line_u == combined.line_pp].sort_values(by='over_prob').tail(15)

,prop_id,player,line_u,line_pp,player_id,stat,event_time,opp,team,count,over_prob,under_prob,pred
1617,697250a7-f,Karl-Anthony Towns,2.5,2.5,43144,AST,2023-11-10 19:00:00-06:00,SAS,MIN,12,0.523189,0.476811,2.766966
1040,ccec37d8-7,Norchad Omier,14.5,14.5,286134,PTS,2023-11-10 18:00:00-06:00,UCF,MIA,1,0.523225,0.476775,14.890592
328,80683233-8,Braden Smith,3.5,3.5,264379,AST,2023-11-10 18:00:00-06:00,MOR,PUR,1,0.525773,0.474227,3.794026
1469,095d3e65-b,Brandon Aiyuk,4.5,4.5,38296,Receptions,2023-11-12 12:00:00-06:00,JAX,SF,16,0.525973,0.474027,4.810366
1495,1cfc277e-4,Ty Berry,1.5,1.5,245703,3P,2023-11-10 19:30:00-06:00,DAY,NW,1,0.526118,0.473882,1.761057
609,e39b1980-9,Josh Giddey,6.5,6.5,42679,TRB,2023-11-10 21:00:00-06:00,SAC,OKC,12,0.526300,0.473700,6.838630
420,9cbb0e1e-7,Boo Buie,19.5,19.5,286940,PTS,2023-11-10 19:30:00-06:00,DAY,NW,1,0.527785,0.472215,19.977229
1233,3e4299ab-8,Joel Embiid,4.5,4.5,43292,AST,2023-11-10 18:00:00-06:00,DET,PHI,12,0.528292,0.471708,4.823068
971,d66f61ba-7,Brandin Cooks,2.5,2.5,38000,Receptions,2023-11-12 15:25:00-06:00,NYG,DAL,15,0.528499,0.471501,2.788938
582,ef699074-4,Kobe Elvis,1.5,1.5,245089,3P,2023-11-10 19:30:00-06:00,NW,DAY,1,0.529804,0.470196,1.773192


In [7]:
fantasy = dog_lines.merge(pp_lines[~pp_lines.league_id.isin([192,84])],how='inner',on=['player','stat'],suffixes=['_dog','_pp'])
fantasy['diff'] = fantasy['line_dog'] - fantasy['line_pp']
fantasy['Z'] = fantasy['diff']/fantasy['line_pp']
fantasy = fantasy[fantasy['diff'] != 0]
fantasy = fantasy[['player','stat','diff','Z','line_dog','line_pp','league_name']].sort_values(by='Z')
fantasy = fantasy.loc[fantasy.league_name != 'NFL SPRINTS']
fantasy

,player,stat,diff,Z,line_dog,line_pp,league_name
1112,Jesus Ferreira,Shots,-0.5,-0.200000,2.0,2.5,SOCCER
1129,369,MAPS 1-3 Kills,-1.0,-0.153846,5.5,6.5,LoL
153,Derrick White,TRB,-0.5,-0.125000,3.5,4.0,NBA
230,Jordan Clarkson,TRB,-0.5,-0.125000,3.5,4.0,NBA
103,Tyus Jones,PTS,-1.0,-0.105263,8.5,9.5,NBA
...,...,...,...,...,...,...,...
1116,Cristian Roldan,Shots,0.5,0.333333,2.0,1.5,SOCCER
1123,ON,MAPS 1-3 Kills,0.5,0.333333,2.0,1.5,LoL
1096,Federico Bonazzoli,Shots,0.5,0.333333,2.0,1.5,SOCCER
1103,Oihan Sancet,Shots,0.5,0.333333,2.0,1.5,SOCCER


In [8]:
pd.set_option('display.max_rows', 33)

In [14]:
combined.loc[combined.player=='Jordan Poole']

,prop_id,player,line_u,line_pp,player_id,stat,event_time,opp,team,count,over_prob,under_prob,pred
0,001cc313-b,Jordan Poole,2.5,2.5,43093,TRB,2023-11-10 18:00:00-06:00,CHA,WAS,12,0.496812,0.503188,2.659030
1249,3b4d6233-e,Jordan Poole,3.5,4.0,43093,AST,2023-11-10 18:00:00-06:00,CHA,WAS,12,0.534051,0.465949,3.859354
1250,3b4d6233-e,Jordan Poole,4.0,4.0,43093,AST,2023-11-10 18:00:00-06:00,CHA,WAS,1,0.500000,0.500000,3.859354
1277,3ad19128-5,Jordan Poole,23.5,21.5,43093,PTS+TRB,2023-11-10 18:00:00-06:00,CHA,WAS,1,0.445860,0.554140,22.119222
1278,3ad19128-5,Jordan Poole,22.5,21.5,43093,PTS+TRB,2023-11-10 18:00:00-06:00,CHA,WAS,1,0.495845,0.504155,22.119222
1279,3ad19128-5,Jordan Poole,21.5,21.5,43093,PTS+TRB,2023-11-10 18:00:00-06:00,CHA,WAS,5,0.514659,0.485341,22.119222
1357,0e006f89-9,Jordan Poole,25.5,26.5,43093,PTS+TRB+AST,2023-11-10 18:00:00-06:00,CHA,WAS,3,0.549335,0.450665,26.660088
1358,0e006f89-9,Jordan Poole,25.5,13.5,43093,PTS+TRB+AST,2023-11-10 18:00:00-06:00,CHA,WAS,3,0.549335,0.450665,26.660088
1359,0e006f89-9,Jordan Poole,26.5,26.5,43093,PTS+TRB+AST,2023-11-10 18:00:00-06:00,CHA,WAS,9,0.505605,0.494395,26.660088
1360,0e006f89-9,Jordan Poole,26.5,13.5,43093,PTS+TRB+AST,2023-11-10 18:00:00-06:00,CHA,WAS,9,0.505605,0.494395,26.660088


In [9]:
all_three = combined.merge(fantasy,on=['player','stat','line_pp'])

In [10]:
all_three.loc[(all_three.line_dog < all_three.line_pp) & (all_three.over_prob > 0.5) & ~all_three.stat.str.contains('Yard')].sort_values(by='over_prob')

,prop_id,player,line_u,line_pp,player_id,stat,event_time,opp,team,count,over_prob,under_prob,pred,diff,Z,line_dog,league_name
116,e094a249-4,Tyrese Maxey,23.5,23.5,42939,PTS,2023-11-10 18:00:00-06:00,DET,PHI,9,0.501647,0.498353,23.574776,-1.0,-0.042553,22.5,NBA
374,603cd827-a,Luka Doncic,38.5,39.0,93,PTS+TRB,2023-11-10 19:30:00-06:00,LAC,DAL,4,0.502843,0.497157,39.008011,-0.5,-0.012821,38.5,NBA
322,7ba20835-5,Jrue Holiday,11.5,11.5,42798,TRB+AST,2023-11-10 18:30:00-06:00,BKN,BOS,3,0.503214,0.496786,11.693804,-0.5,-0.043478,11.0,NBA
104,e90f74b1-9,Jaylen Brown,31.5,31.5,242292,PTS+TRB+AST,2023-11-10 18:30:00-06:00,BKN,BOS,9,0.503624,0.496376,31.730835,-1.0,-0.031746,30.5,NBA
343,73b42e6c-f,Rudy Gobert,27.5,27.0,42685,PTS+TRB+AST,2023-11-10 19:00:00-06:00,SAS,MIN,5,0.504302,0.495698,27.640610,-0.5,-0.018519,26.5,NBA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,5e05c38a-2,Shai Gilgeous-Alexander,5.5,6.0,43395,AST,2023-11-10 21:00:00-06:00,SAC,OKC,10,0.550996,0.449004,6.007315,-0.5,-0.083333,5.5,NBA
99,95cdce27-9,Derrick White,3.5,4.0,42962,TRB,2023-11-10 18:30:00-06:00,BKN,BOS,6,0.551519,0.448481,3.981322,-0.5,-0.125000,3.5,NBA
76,85345981-3,Jaylen Brown,21.5,22.0,242292,PTS,2023-11-10 18:30:00-06:00,BKN,BOS,2,0.553293,0.446707,22.334238,-0.5,-0.022727,21.5,NBA
115,e094a249-4,Tyrese Maxey,22.5,23.5,42939,PTS,2023-11-10 18:00:00-06:00,DET,PHI,4,0.554180,0.445820,23.574776,-1.0,-0.042553,22.5,NBA


In [11]:
all_three

,prop_id,player,line_u,line_pp,player_id,stat,event_time,opp,team,count,over_prob,under_prob,pred,diff,Z,line_dog,league_name
0,a659a923-1,Ivica Zubac,19.0,19.0,42545,PTS+TRB,2023-11-10 19:30:00-06:00,DAL,LAC,1,0.500000,0.500000,19.519369,0.5,0.026316,19.5,NBA
1,a659a923-1,Ivica Zubac,19.5,19.0,42545,PTS+TRB,2023-11-10 19:30:00-06:00,DAL,LAC,6,0.492045,0.507955,19.519369,0.5,0.026316,19.5,NBA
2,a59dcc19-9,Breece Hall,18.5,17.5,38803,Receiving Yards,2023-11-12 19:20:00-06:00,LV,NYJ,14,0.498416,0.501584,18.584652,1.0,0.057143,18.5,NFL
3,a59dcc19-9,Breece Hall,17.5,17.5,38803,Receiving Yards,2023-11-12 19:20:00-06:00,LV,NYJ,1,0.500000,0.500000,18.584652,1.0,0.057143,18.5,NFL
4,a5293ca6-5,Jordan Clarkson,3.5,4.0,43106,TRB,2023-11-10 19:00:00-06:00,MEM,UTA,11,0.531426,0.468574,3.849691,-0.5,-0.125000,3.5,NBA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,5e1cc3d3-d,Domantas Sabonis,13.5,13.0,240737,TRB,2023-11-10 21:00:00-06:00,OKC,SAC,7,0.479523,0.520477,13.345916,0.5,0.038462,13.5,NBA
382,56d54772-4,Austin Ekeler,35.5,36.5,35465,Receiving Yards,2023-11-12 15:05:00-06:00,DET,LAC,9,0.507090,0.492910,36.660354,-3.0,-0.082192,33.5,NFL
383,56d54772-4,Austin Ekeler,36.5,36.5,35465,Receiving Yards,2023-11-12 15:05:00-06:00,DET,LAC,2,0.500000,0.500000,36.660354,-3.0,-0.082192,33.5,NFL
384,56d54772-4,Austin Ekeler,37.5,36.5,35465,Receiving Yards,2023-11-12 15:05:00-06:00,DET,LAC,5,0.505091,0.494909,36.660354,-3.0,-0.082192,33.5,NFL


In [12]:
pd.set_option('display.max_rows',35)